In [ ]:
from docplex.mp.model import Model
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import distance_matrix as dm
import pandas as pd
import os

TARGET_WEEKDAY = 0  # 0 = Monday, 1 = Tuesday, ..., 6 = Sunday
TARGET_HOUR = 8  # 0 = 00:00, 1 = 01:00, ..., 23 = 23:00

load_dotenv()

api_key = os.getenv("GOOGLE_MAPS_API_KEY")

matrix_time_first_tw, matrix_distance = dm.main(TARGET_WEEKDAY, TARGET_HOUR, api_key)
matrix_time_second_tw, matrix_distance = dm.main(
    TARGET_WEEKDAY, TARGET_HOUR + 1, api_key
)
matrix_time_third_tw, matrix_distance = dm.main(
    TARGET_WEEKDAY, TARGET_HOUR + 2, api_key
)
matrix_time_fourth_tw, matrix_distance = dm.main(
    TARGET_WEEKDAY, TARGET_HOUR + 3, api_key
)

print(matrix_time_first_tw)
print("\n\n")
print(matrix_time_second_tw)
print("\n\n")
print(matrix_time_third_tw)
print("\n\n")
print(matrix_time_fourth_tw)
print("\n\n")
print(matrix_distance)

# sava datas in a matrix of matrixes
first_data = matrix_time_first_tw
second_data = matrix_time_second_tw
third_data = matrix_time_third_tw
fourth_data = matrix_time_fourth_tw

data = [first_data, second_data, third_data, fourth_data]

# parameters for time model
c = 10  # number of nodes
d = [8, 9, 10, 11, 12]  # time intervals
tw = [
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
]  # time windows
q = [
    0,
    10,
    10,
    -10,
    20,
    10,
    -10,
    20,
    40,
    -30,
]  # quantity of goods to collect/deliver
Q = 100  # maximum capacity of the vehicle
t_mv = 0.5  # time to move one item in/out from a vehicle [min]
M = 100000  # big M
tot_q = 0  # total quantity of goods to collect/delivery
del_q = 0  # quantity of goods to deliver
v = 3
# parameters for energy model
distances = matrix_distance  # should match value of c
print(distances)
A = [(i, j) for i in range(c) for j in range(c) if i != j]
N0 = list(range(1, c))  # Nodi clienti (escluso il deposito)
L = 100  # Capacità massima della batteria
a = {(i, j): 2 for (i, j) in A}  # Consumo base per arco
b = {(i, j): 0.5 for (i, j) in A}  # Consumo dipendente dal carico
c_E = 1  # Costo per unità di energia consumata
c_F = 1  # Costo fisso per veicolo
c_M = 1  # Costo per unità di distanza

unused_vehicle = False

for good in q:
    tot_q += abs(good)

tot_t_mv = tot_q * t_mv

print("Total time required to collect/deliver all goods:", tot_t_mv, "minutes")

for good in q:
    if good < 0:
        del_q += good
del_q = abs(del_q)

print(f"Computing for {v} vehicles")

# define model
mdl = Model("CVRP")
mdl.parameters.timelimit = 30

# decision variables

x = mdl.binary_var_dict(
    (
        (i, j, t, k)
        for i in range(c)
        for j in range(c)
        for t in range(len(d) - 1)
        for k in range(v)
    ),
    name="x",
)  # x == 1 if it is true that we are going from node i to node j at time t with vehicle k
T = mdl.continuous_var_matrix(
    c + 1, v, name="t"
)  # arrival time at node i for vehicle k
C = mdl.continuous_var_matrix(
    c, v, name="C"
)  # quantity of goods when you leave node i for vehicle k

# nodes constraints
for k in range(v):
    mdl.add_constraint(
        mdl.sum(mdl.sum(x[0, j, t, k] for j in range(1, c)) for t in range(len(d) - 1))
        <= 1  # TO CHANGE FROM ==(it was a test to make boot vehicle works) to <=
    )  # every vehicle must leave the depot exactly once, hence every vehicle is expoloited

for k in range(v):
    mdl.add_constraint(
        mdl.sum(mdl.sum(x[j, 0, t, k] for j in range(1, c)) for t in range(len(d) - 1))
        == mdl.sum(
            mdl.sum(x[0, j, t, k] for j in range(1, c)) for t in range(len(d) - 1)
        )
    )  # every vehicle that has left the depot must return to it

for j in range(1, c):
    mdl.add_constraint(
        mdl.sum(
            mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c) if i != j)
                for t in range(len(d) - 1)
            )
            for k in range(v)
        )
        == 1
    )  # every node must be visited exactly once

for k in range(v):
    for j in range(1, c):
        mdl.add_constraint(
            mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c) if i != j)
                for t in range(len(d) - 1)
            )
            == mdl.sum(
                mdl.sum(x[j, i, t, k] for i in range(c) if i != j)
                for t in range(len(d) - 1)
            )
        )  # when you visit a node you ought to leave it

# time
for k in range(v):
    for t in range(len(d) - 1):
        mdl.add_constraint(
            T[0, k] >= d[t] * 60 * mdl.sum(x[0, j, t, k] for j in range(1, c))
        )  # when you leave depot at a time t, you ought to leave it at that time

for k in range(v):
    for t in range(len(d) - 1):
        for j in range(1, c):
            mdl.add_constraint(
                T[j, k] >= d[t] * 60 * mdl.sum(x[i, j, t, k] for i in range(1, c))
            )  # when you arrive at a node j from node i at time t, you ought to arrive at that time

for k in range(v):
    for t in range(len(d) - 1):
        for j in range(1, c):
            mdl.add_constraint(
                T[j, k]
                <= d[t + 1] * 60 + (1 - mdl.sum(x[i, j, t, k] for i in range(1, c))) * M
            )  # when arrive in j from i at time t, you ought to reach it before the end of the time interval

for k in range(v):
    for t in range(len(d) - 1):
        for i in range(1, c):
            mdl.add_constraint(
                T[i, k] >= d[t] * 60 * mdl.sum(x[i, j, t, k] for j in range(0, c))
            )  # when you arrive at node j from node i at time t, you ought to depart in that time interval

for k in range(v):
    for t in range(len(d) - 1):
        for i in range(1, c):
            mdl.add_constraint(
                T[i, k]
                <= d[t + 1] * 60 + (1 - mdl.sum(x[i, j, t, k] for j in range(0, c))) * M
            )  # when you arrive in j from i at time t, you ought to depart from i after the start of that time interval

for k in range(v):
    for i in range(c):
        for j in range(1, c):
            for t in range(len(d) - 1):
                if i != j:
                    mdl.add_constraint(
                        (1 - x[i, j, t, k]) * M + T[j, k] - T[i, k]
                        >= data[t][i][j] + t_mv * abs(q[j])
                    )
                    mdl.add_constraint(
                        (1 - x[i, j, t, k]) * M - T[j, k] + T[i, k]
                        >= -data[t][i][j] - t_mv * abs(q[j])
                    )  # if you go from i to j, then the difference between the arrival time at j and i should be the same as the time required to go from i to j
                    # plus the time to load/unload the required goods

for k in range(v):
    for t in range(len(d) - 1):
        for i in range(1, c):
            mdl.add_constraint(
                (1 - x[i, 0, t, k]) * M + T[c, k] - T[i, k] >= data[t][i][0]
            )
            mdl.add_constraint(
                (1 - x[i, 0, t, k]) * M - T[c, k] + T[i, k] >= -data[t][i][0]
            )  # if you go from i to the depot, then the difference between the arrival time at the depot and i should be the same as the time required to go
            # from i to the depot

for k in range(v):
    for i in range(1, c):
        mdl.add_constraint(
            tw[i - 1][0] * 60 <= T[i, k]
        )  # you must arrive in i on-time (based on client side) lowest limit
        mdl.add_constraint(
            T[i, k] <= tw[i - 1][1] * 60
        )  # you must arrive in i on-time (based on client side) upper limit

for k in range(v):
    mdl.add_constraint(
        T[c, k] - T[0, k] <= 8 * 60
    )  # you must return to the depot within 8 hours (a shift)

# quantity constraints
for k in range(v):
    for t in range(len(d) - 1):
        for i in range(c):
            for j in range(1, c):
                if i != j:
                    mdl.add_constraint(
                        C[i, k] - C[j, k] + q[j] <= (1 - x[i, j, t, k]) * M
                    )
                    mdl.add_constraint(
                        C[j, k] - C[i, k] - q[j] <= (1 - x[i, j, t, k]) * M
                    )  # if you go from i to j, then the difference between the quantity of goods at i and j should be the same as the quantity
                    # of goods to deliver at j

for k in range(v):
    for i in range(c):
        mdl.add_constraint(
            max(0, q[i]) <= C[i, k]
        )  # if node i requires to load goods, you must leave i with at least the quantity of goods to collect
        mdl.add_constraint(
            C[i, k] <= min(Q, Q + q[i])
        )  # you must leave i with at most Q+q[i] goods if you have to deliver at node i goods or Q if you have to collect goods at node i

for k in range(v):
    del_qi = 0
    for j in range(1, c):
        if q[j] < 0:
            del_qi += q[j] * mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c)) for t in range(len(d) - 1)
            )
    mdl.add_constraint(
        C[0, k] >= -del_qi
    )  # each vehicle must depart from the depot with at least the quantity of goods to deliver for its route

# objective function
mdl.minimize(
    mdl.sum(
        mdl.sum(
            mdl.sum(
                mdl.sum(x[i, j, t, k] * data[t][i][j] for i in range(c))
                for j in range(c)
            )
            for t in range(len(d) - 1)
        )
        for k in range(v)
    )
)  # minimize time of routes

sol = mdl.solve()

if sol is None:
    print("No solution found")

print("\n\n\nTotal time required is:")

print(sol.objective_value)

print(sol.objective_value + tot_t_mv)

print("\n\n\n")

times.append(sol.objective_value + tot_t_mv)
for k in range(v):
    counter = 0
    for t in range(len(d) - 1):
        for i in range(c):
            for j in range(c):
                if sol.get_value(x[i, j, t, k]) == 1:
                    counter += 1
    if counter != 0:
        print("vehicle ", k, "is used")
    if counter == 0:
        print("vehicle ", k, "is NOT used")


# for k in range(v):
#     print(f"\nVehicle {k}:\n")
#     for t in range(len(d) - 1):
#         for row in range(c):
#             for col in range(c):
#                 if row == 0 and sol.get_value(x[row, col, t, k]) == 1:
#                     route = [(t, row, sol.get_value(T[row, k]))]
#                     capacity = [(row, sol.get_value(C[row, k]))]
#                     route.append((t, col, sol.get_value(T[col, k])))
#                     capacity.append((col, sol.get_value(C[col, k])))
#                     print(f" (riga: {row:.2f} , colonna: {col:.2f}range temporale:{t})\n")
#                 elif col == 0 and sol.get_value(x[row, col, t, k]) == 1:
#                     end = (t, col, sol.get_value(T[c, k]))
#                     print(f" (riga: {row:.2f} , colonna: {col:.2f}range temporale:{t})\n")
#                 elif sol.get_value(x[row, col, t, k]) == 1:
#                     route.append((t, col, sol.get_value(T[col, k])))
#                     capacity.append((col, sol.get_value(C[col, k])))
#                     print(f" (riga: {row:.2f} , colonna: {col:.2f}range temporale:{t})\n")
#     route.append(end)

#     print(
#         " -> ".join(
#             f"{node} (arrivo: {arrival:.2f} , range temporale:{t})"
#             for t, node, arrival in route
#         )
#     )

#     print("\nCapacity for each node in order of visit:")
#     for node, cap in capacity:
#         print(f"Node {node}: {cap:.2f}")

# Creazione del modello

[[ 0.   19.83 17.05 19.62 23.92 22.42 12.47 20.5  12.47 21.32]
 [ 9.43  0.   11.13 13.62 18.42 16.82 15.68 26.9  11.3  22.48]
 [24.72 20.27  0.    8.93  7.77  5.98 28.52 40.87 22.82 10.58]
 [20.77 12.87  5.97  0.    4.92  5.15 21.28 37.35 19.72 14.62]
 [19.42 29.63 26.97 28.25  0.   26.37 28.88 34.62 25.35 17.1 ]
 [21.   13.13  6.2   3.32  5.23  0.   21.33 37.98 19.98 14.85]
 [13.43 23.9  19.48 23.08 27.   25.    0.   27.4   8.48 31.28]
 [19.68 36.23 32.97 36.47 41.25 39.9  24.12  0.   30.52 35.25]
 [14.55 19.15 11.75 19.12 19.28 17.48  7.6  32.3   0.   22.68]
 [22.13 22.6  14.87 12.6   8.83  6.98 31.12 37.4  25.35  0.  ]]



[[ 0.   20.63 17.22 20.47 23.82 22.5  13.17 18.8  11.33 19.45]
 [ 9.78  0.   10.87 13.95 17.97 16.53 15.3  23.28 11.25 20.52]
 [24.83 20.57  0.    9.25  7.63  5.87 27.18 37.2  22.1  10.38]
 [21.75 13.87  6.28  0.    5.02  5.25 21.08 34.25 19.77 13.07]
 [20.07 30.68 27.8  28.73  0.   25.12 29.   31.4  26.4  16.38]
 [21.48 13.65  6.13  3.68  5.15  0.   20.85 34.27 1

NameError: name 'mdl' is not defined

In [1]:
from docplex.mp.model import Model
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import distance_matrix as dm
import pandas as pd
import os

TARGET_WEEKDAY = 0  # 0 = Monday, 1 = Tuesday, ..., 6 = Sunday
TARGET_HOUR = 8  # 0 = 00:00, 1 = 01:00, ..., 23 = 23:00

load_dotenv()

api_key = os.getenv("GOOGLE_MAPS_API_KEY")

matrix_times, matrix_distance = dm.main(TARGET_WEEKDAY, TARGET_HOUR, api_key)

# parameters for time model
c = 10  # number of nodes
q = [
    0,
    10,
    10,
    -10,
    20,
    10,
    -10,
    20,
    40,
    -30,
]  # quantity of goods to collect/deliver
Q = 100  # maximum capacity of the vehicle
distances = matrix_distance  # should match value of c
A = [(i, j) for i in range(c) for j in range(c) if i != j]
N0 = list(range(1, c))  # Nodi clienti (escluso il deposito)
L = 100  # Capacità massima della batteria
a = {(i, j): 2 for (i, j) in A}  # Consumo base per arco
b = {(i, j): 0.5 for (i, j) in A}  # Consumo dipendente dal carico
c_E = 1  # Costo per unità di energia consumata
c_F = 1  # Costo fisso per veicolo
c_M = 1  # Costo per unità di distanza

v = 3  # Number of vehicles

for i in range(len(distances)):
    for j in range(len(distances)):
        distances[i][j] = distances[i][j] / 1000  # Convert meters to kilometers

mdl_energy = Model("EnergyOptimization")

# Variabili di decisione

x_energy = mdl_energy.binary_var_cube(
    len(distances), len(distances), v, name="x"
)  # Percorsi
f = mdl_energy.continuous_var_cube(
    len(distances), len(distances), v, name="f"
)  # Flussi di carico
w = mdl_energy.continuous_var_list(v, name="w")  # Consumo massimo di energia

# Funzione obiettivo: Minimizzare i costi energetici

mdl_energy.minimize(
    mdl_energy.sum(
        [
            mdl_energy.sum([c_F * x_energy[0, j, k] for j in range(1, len(distances))])
            + mdl_energy.sum([c_M * distances[i][j] for i, j in A])
            + c_E * w[k]
            for k in range(v)
        ]
    )
)

# Vincoli
for k in range(v):
    mdl_energy.add_constraint(mdl_energy.sum(x_energy[0, j, k] for j in N0) <= 1)

for i in N0:
    mdl_energy.add_constraint(
        mdl_energy.sum(
            x_energy[i, j, k] for k in range(v) for j in range(len(distances)) if j != i
        )
        == 1
    )

for i in range(len(distances)):
    mdl_energy.add_constraint(
        mdl_energy.sum(x_energy[j, i, k] for j in range(len(distances)) if j != i)
        == mdl_energy.sum(x_energy[i, j, k] for j in range(len(distances)) if j != i)
    )

for i, j in A:
    for k in range(v):
        mdl_energy.add_constraint(q[j] * x_energy[i, j, k] <= f[i, j, k])
        mdl_energy.add_constraint(f[i, j, k] <= (Q - q[i]) * x_energy[i, j, k])

for i in N0:
    mdl_energy.add_constraint(
        mdl_energy.sum(
            f[j, i, k] for k in range(v) for j in range(len(distances)) if j != i
        )
        - mdl_energy.sum(
            f[i, j, k] for k in range(v) for j in range(len(distances)) if j != i
        )
        == q[i]
    )

for k in range(v):
    mdl_energy.add_constraint(
        mdl_energy.sum(a[i, j] * x_energy[i, j, k] + b[i, j] * f[i, j, k] for i, j in A)
        <= w[k]
    )

for k in range(v):
    mdl_energy.add_constraint(0 <= w[k])
    mdl_energy.add_constraint(w[k] <= L)

# Risoluzione del modello

solution = mdl_energy.solve(log_output=False)

cost = solution.objective_value

if solution is None:
    print("Il modello non ha soluzione")
else:
    print("Costo totale:", solution.objective_value)
    for k in range(v):
        print(
            f"Consumo massimo del veicolo {k}: {solution.get_value(mdl_energy.get_var_by_name(f'w_{k}'))}"
        )

w = solution.get_values(w)

for w in w:
    if w == 0:
        unused_vehicle = True
        print("\nEsistono uno o più veicoli che non vengono utilizzati")
        break

Costo totale: 1240.0320000000006
Consumo massimo del veicolo 0: 41.0
Consumo massimo del veicolo 1: 17.0
Consumo massimo del veicolo 2: 24.0
